In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import sys

In [2]:
rating = pd.read_csv('../data/ratings.csv')
books = pd.read_csv('../data/books.csv')

In [3]:
books = books[['id','book_id','authors','original_title','language_code','average_rating','ratings_count']]

In [4]:
rating['book_id'] = rating['book_id']-1
rating['user_id'] = rating['user_id']-1
books['id'] = books['id']-1

In [5]:
rating.drop_duplicates(inplace=True)
rating = rating.reset_index(drop=True)

In [6]:
num_user = len(rating['user_id'].unique())
num_book = len(rating['book_id'].unique())

In [7]:
# Create a sparse iteration matrix from rating data
def create_sparse_matrix(num_user,num_item,df):
    indices = df[['user_id','book_id']].values.astype(np.int32)
    values = df['rating'].values.astype(np.int32)
    dense_shape = (num_user,num_item)
    resp =  tf.SparseTensor(indices,values,dense_shape)
    return resp

In [8]:
rating_matrix = create_sparse_matrix(num_user,num_book,rating)
rating_matrix = tf.Session().run(rating_matrix)

In [9]:
sparsity = 1-len(rating_matrix.values)/np.prod(rating_matrix.dense_shape)
print("Sparsity of rating matrix:",sparsity)

Sparsity of rating matrix: 0.9981654088050315


In [10]:
observed_user = rating_matrix.indices[:,0]
observed_item = rating_matrix.indices[:,1]
observed_rating = np.ones_like(observed_user)

#### Negative sampling un-observed items

In [11]:
# Get the number of book each user has read to sample accordingly
users,books_read = rating.groupby('user_id')['book_id'].agg('count').reset_index().values.T

In [12]:
alpha = 4 # For each observed entry, samples 4 negative entries
unobserved_user = []
unobserved_item = []
for u,k in zip(users,books_read):
    neg_sample = np.random.choice(num_book,size=alpha*k,replace=False)
    neg_user = u*np.ones_like(neg_sample)
    unobserved_user += neg_user.tolist()
    unobserved_item += neg_sample.tolist()
unobserved_user = np.array(unobserved_user)
unobserved_item = np.array(unobserved_item)
unobserved_rating = np.zeros_like(unobserved_item)

In [25]:
train_user = np.concatenate([observed_user,unobserved_user])
train_item = np.concatenate([observed_item,unobserved_item])
train_rating = np.concatenate([observed_rating,unobserved_rating])

In [26]:
tf.reset_default_graph()

In [27]:
class NeuralCF:
    def __init__(self,mf_latent,mlp_latent,mlp_layers,num_user,num_item):
        self.mf_latent = mf_latent
        self.mlp_latent = mlp_latent
        self.mlp_layers = mlp_layers
        self.num_user = num_user
        self.num_item = num_item
        self._create_mf_model()

    def _create_mf_model(self):
        user_input = tf.keras.layers.Input(shape=1,name='user_input')
        item_input = tf.keras.layers.Input(shape=1,name='item_input')
        
        mf_user_latent = tf.keras.layers.Embedding(input_dim=self.num_user,
                                                   output_dim=self.mf_latent,
                                                   embeddings_initializer='normal',
                                                   input_length=1,
                                                   name='MF_user')(user_input)
        mf_item_latent = tf.keras.layers.Embedding(input_dim=self.num_item,
                                                   output_dim=self.mf_latent,
                                                   embeddings_initializer='normal',
                                                   input_length=1,
                                                   name='MF_item')(item_input)
        
        mlp_user_latent = tf.keras.layers.Embedding(input_dim=self.num_user,
                                                    output_dim=self.mlp_latent,
                                                    embeddings_initializer='normal',
                                                    input_length=1,
                                                    name='MLP_user')(user_input)
        mlp_item_latent = tf.keras.layers.Embedding(input_dim=self.num_item,
                                                    output_dim=self.mlp_latent,
                                                    embeddings_initializer='normal',
                                                    input_length=1,
                                                    name='MLP_item')(item_input)
        
        
        flatten_mf_user = tf.keras.layers.Flatten()(mf_user_latent)
        flatten_mf_item = tf.keras.layers.Flatten()(mf_item_latent)
        
        def element_wise_prod(values):
            return tf.multiply(values[0],values[1])
        
        mf_out = tf.keras.layers.Lambda(element_wise_prod,name='mf_out')([flatten_mf_user,flatten_mf_item])
        
        def concatenate(values):
            return tf.concat(values,axis=-1)
        
        flatten_mlp_user = tf.keras.layers.Flatten()(mlp_user_latent)
        flatten_mlp_item = tf.keras.layers.Flatten()(mlp_item_latent)
        
        mlp = tf.keras.layers.Lambda(concatenate,name='mlp_concat')([flatten_mlp_user,flatten_mlp_item])
        for i,l in enumerate(self.mlp_layers[:-1]):
            mlp = tf.keras.layers.Dense(l, activation='selu',
                                        kernel_initializer='glorot_normal',
                                        name = f'MLP_dense_{i}')(mlp)
            
        mlp_out = tf.keras.layers.Dense(self.mlp_layers[-1],
                                        activation='selu',
                                        kernel_initializer='glorot_normal',
                                        name='mlp_out')(mlp)
        self.mf_model = tf.keras.models.Model(inputs=[user_input,item_input],outputs=[mf_out])
        self.mlp_model = tf.keras.models.Model(inputs=[user_input,item_input],outputs=[mlp_out])
        
        mf_mlp_concat = tf.keras.layers.Lambda(concatenate,name='mf_mlp_concat')([mf_out,mlp_out])
        final_out = tf.keras.layers.Dense(1, activation='sigmoid',
                                          kernel_initializer='glorot_normal',
                                          name='prediction')(mf_mlp_concat)
        
        self.neucf_model = tf.keras.models.Model(inputs=[user_input,item_input],outputs=[final_out])
        
    def compiles(self):
        return
    def train(self):
        return
    def predict(self):
        return

In [28]:
neucf = NeuralCF(64,64,[256,128,64],num_user,num_book)

W0726 14:07:00.248401 4581770688 deprecation.py:506] From /Users/trungdoan/.conda/envs/myenv/lib/python3.7/site-packages/tensorflow/python/keras/initializers.py:143: calling RandomNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0726 14:07:00.358266 4581770688 deprecation.py:506] From /Users/trungdoan/.conda/envs/myenv/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1288: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0726 14:07:00.392729 4581770688 deprecation.py:323] From /Users/trungdoan/.conda/envs/myenv/lib/python3.7/site-packages/tensorflow/python/keras/backend.py:4075: add_

In [29]:
neucf.neucf_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

In [129]:
neucf.neucf_model.fit(x=[train_user,train_item],y=train_rating,batch_size=1024,epochs=5,)

Epoch 1/5
4900560/4900560 [==============================] - 268s 55us/sample - loss: 0.0550 - acc: 0.9813
Epoch 2/5
4900560/4900560 [==============================] - 269s 55us/sample - loss: 0.0451 - acc: 0.9853
Epoch 3/5
 236544/4900560 [>.............................] - ETA: 4:15 - loss: 0.0257 - acc: 0.9925

KeyboardInterrupt: 

In [130]:
def recommend(userid,k=10):
    target_user = userid
    target_book = np.arange(0,num_book)
    target_arr = target_user*np.ones_like(target_book)
    preds = neucf.neucf_model.predict(x=[target_arr,target_book])
    preds = preds.reshape(-1)
    recom_books = np.argsort(preds)[::-1][:k]
    return recom_books,preds

### Model Evaluation

In [131]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Coverages
coverages = []
recom_probs = []
SAMPLE_SIZE = 5000
test_sample = np.random.choice(num_user,replace=False,size=SAMPLE_SIZE)
for c,i in enumerate(test_sample):
    sys.stdout.write(f'\rRecommend: {c}/{SAMPLE_SIZE}')
    sys.stdout.flush()
    recom,probs = recommend(i)
    coverages.append(recom)
    recom_probs.append(probs)

Recommend: 3288/5000

In [152]:
unique_coverages = np.unique(np.concatenate(coverages))
read_books = rating[rating['user_id'].isin(test_sample)]['book_id'].values
unique_read_books = np.unique(read_books)
print(f"Sampled test users cover {len(unique_read_books)/num_book*100:.02f}% of the data")
print(f"The recommendation cover {len(unique_coverages)/num_book*100:.02f}% of the data")
print(f"On average, each users read {len(read_books)/SAMPLE_SIZE} books")
print("==================================")

recom_probs = np.array(recom_probs)
similarity = cosine_similarity(recom_probs)
# Personalized Score = 1 - mean_of_upper_triangle(similarity)
# The higher the score, the better
personalized_score = 1-np.sum(np.triu(similarity))/(np.prod(similarity.shape)/2)
print("Personalization Score =",personalized_score)

print("==================================")
read_book_by_user = rating.groupby('user_id')['book_id'].agg(lambda x: list(x)).reset_index()
read_book_by_user = read_book_by_user[read_book_by_user['user_id'].isin(test_sample)]['book_id'].values
new_books = []
for i,j in zip(coverages,read_book_by_user):
    num_new_books = len([v for v in i if v not in j])
    new_books.append(num_new_books)
print(f"maximum number of recommended new books: {np.max(new_books)}")
print(f"minimum number of recommended new books: {np.min(new_books)}")
print(f"On average, the sys recommend {np.sum(new_books)/SAMPLE_SIZE} new books per user")

print("==================================")
recall_books = []
for i,j in zip(coverages,read_book_by_user):
    num_recall_books = len(np.intersect1d(i,j))
    recall_books.append(num_recall_books)
print(f"maximum number of recall books: {np.max(recall_books)}")
print(f"minimum number of recall books: {np.min(recall_books)}")
print(f"On average, the sys recall {np.sum(recall_books)/SAMPLE_SIZE} books per user")

Sampled test users cover 84.46% of the data
The recommendation cover 49.67% of the data
On average, each users read 18.144 books
Personalization Score = 0.852988078125
maximum number of recommended new books: 10
minimum number of recommended new books: 7
On average, the sys recommend 9.989 new books per user
maximum number of recall books: 3
minimum number of recall books: 0
On average, the sys recall 0.011 books per user


### Inference